# Advanced free energy analyses with SOMD

In this notebook we will explore advanced analysis techniques for alchemical free enregy calculations, using Sire tool `analyse_freenrg mbar`. 
Initially, you will learn how to correctly interpret phase space overlap matrix. Following, advanced commands will be shown to enhance the mbar analysis.

The notebook forms part of the CCPBio-Sim workshop **Alchemical Free Energy Simulation Analysis with analyse_freenrg** run on the 11th of April 2018 at the University of Bristol.

*Author: Anotnia Mey & Stefano Bosisio  
Email: antonia.mey@ed.ac.uk*

**Reading time of the document: xx mins**

## Overlap matrix

In [ ]:
%pylab inline
import glob
import seaborn as sbn
sbn.set_style("ticks")
sbn.set_context("notebook", font_scale = 2)

### The overlap matrix

can be used to look at the phase space overlap of neighbouring lambdas. 
By adding the flag `--overlap` this matrix will be automatically computed and added to the output file. 

So let's look at the overlap matrix for a simulation of a host-guest system, shown in figure, obtained by running 16 $\lambda$ windows of 8 ns length each. 
This time we will write an output file called `good_overlap.dat`


<img src="images/CBC-G2.png" alt="host-guest" style="width: 300px;"/>


In [ ]:
%%capture run_info_good
#Let's run the analysis again with the keyword --overlap
!~/sire_2018/sire.app/bin/analyse_freenrg mbar -i good_overlap/lambda-*/simfile.dat -o good_overlap.dat --subsampling --overlap

In [ ]:
#A helper function to read the overlap matrix from file
def get_overlp_matrix(filename, lambda_val ):
    fh = open (filename, 'r')
    lines = fh.readlines()
    fh.close()
    count = 0
    matrix = []
    for line in lines:
        if line.startswith('#Overlap'):
            matrix = lines[(count+1):(count+1+lambda_vals)]
            break
        count = count+1 
    for i in range(len(matrix)):
        temp = matrix[i].strip().split(' ')
        float_temp = [float(j) for j in temp]
        matrix[i] = float_temp
    matrix =np.array(matrix)
    return matrix

In [ ]:
#an exercise could be add the lambda_vals in the function above
good_overlap = get_overlp_matrix('good_overlap.dat',16)


### Plotting the overlap matrix
The plotting library has a nice advanced heat map feature that allows you to not only plot a pictorial image of a matrix or heatmap but also add the numercal values making it easier to read the plot

In [ ]:
fig = figure(figsize=(12,12))
ax = sbn.heatmap(good_overlap, annot=True, fmt='.2f', linewidths=.5, annot_kws={"size": 12})
ax.set_xlabel(r'$\lambda$ index')
ax.set_ylabel(r'$\lambda$ index')
ax.set_title('Good overlap matrix')

### Example of a bad overlap matrix
Below we have the same simulation as before, but reducing the number of lambda windows from 11 to 6. What do you observe in terms of the overlap matrix?

Belowe we have the same simulation as before, but the number of $\lambda$ windows was reduced from 16 to 10. What do you observe in terms of the overlap matrix? 
Repeat the procedure above and create a file called `bad_overlap.dat`

In [ ]:
%%capture run_info_bad
#Let's run the analysis again with the keyword --overlap


In [ ]:
bad_overlap = get_overlp_matrix('bad_overlap.dat',10)

In [ ]:
fig = figure(figsize=(12,12))
ax = sbn.heatmap(good_overlap, annot=True, fmt='.2f', linewidths=.5, annot_kws={"size": 12})
ax.set_xlabel(r'$\lambda$ index')
ax.set_ylabel(r'$\lambda$ index')
ax.set_title('Bad overlap matrix')

### Advanced tasks

Now, let's see how we can improve the free energy estimation using the mbar commands `subsampling` and `discard`. The first one will perform a subsampling operation over all the samples data contained in the `simfile.dat` files. The second option will discard a number of frams from the start of the simulation. 

We are going to study a host-guest system, similar to the one shown before. To compute the binding free energy, the following thermodynamic cycle is adopted:
<img src="images/host_guest_cyclo.png" alt="host-guest" style="width: 500px;"/>
Initially, a `discharging` step is performed, where ligand's charges are turned off both in solvated and  bound phases. Following, a `vanishing` step is done, by  switching off ligand's Lennard Jones terms, in order to have a fully decouple molecule. 
The final free energy of binding is computed, by summing up the contribution of each leg of the cycle, as:
\begin{equation}
\Delta G_\mathrm{bind} = (\Delta G^\mathrm{solv}_\mathrm{elec} + \Delta G^\mathrm{solv}_\mathrm{vdW}) - (\Delta G^\mathrm{host}_\mathrm{elec} + \Delta G^\mathrm{host}_\mathrm{vdW} ) 
\end{equation}

As advanced tasks try to perform these steps:
1. Compute the discharging and vanishing free energy for the solvated and bound phase using the standard `analyse_freenrg mbar` command. Thus, retrieve the binding free energy using the equation above
2. Compute the discharging and vanishing free energy by adding the option `--subsampling`. What do you notice? What is happening to TI? and MBAR? What is the final binding free energy?
3. This time try to use just a percentage of the total number of samples, adding `--percent ` option. What is the final outcome?


In [ ]:
#1. Compute the discharging and vanishing free energy for the solvated and bound phase:
#bound phase -discharging
!~/sire_2018/sire.app/bin/analyse_freenrg mbar -i subsampling/bound/run001/discharge/output/lambda-*/simfile.dat -o bound_discharge.dat
#bound phase -vanishing
!~/sire_2018/sire.app/bin/analyse_freenrg mbar -i subsampling/bound/run001/vanish/output/lambda-*/simfile.dat -o bound_vanish.dat
#solvated phase -discharging
!~/sire_2018/sire.app/bin/analyse_freenrg mbar -i subsampling/free/run001/discharge/output/lambda-*/simfile.dat -o free_discharge.dat
#solvated phase -vanishing
!~/sire_2018/sire.app/bin/analyse_freenrg mbar -i subsampling/free/run001/vanish/output/lambda-*/simfile.dat -o free_vanish.dat


In [ ]:
#helper function to extract DG from the freenrg_analysis generated files. 
def get_free_energy(ifile):
    reader = open(ifile,"r").readlines()
    mbar = float(reader[-3].split(",")[0])
    ti = float(reader[-1].split()[0])
    return mbar, ti

In [ ]:
#Extract the free enery change from the output files
bound_discharge_mbar, bound_discharge_ti = get_free_energy('bound_discharge.dat')
bound_vanish_mbar,bound_vanish_ti = get_free_energy('bound_vanish.dat')

free_discharge_mbar,free_discharge_ti = get_free_energy('free_discharge.dat')
free_vanish_mbar, free_vanish_ti = get_free_energy('free_vanish.dat')

#mbar
DG_bound_mbar =bound_discharge_mbar + bound_vanish_mbar
DG_free_mbar = free_discharge_mbar + free_vanish_mbar
#ti
DG_bound_ti = bound_discharge_ti + bound_vanish_ti
DG_free_ti = free_discharge_ti + free_vanish_ti


#DGbind
DG_bind_mbar = DG_free_mbar - DG_bound_mbar
DG_bind_ti =  DG_free_ti - DG_bound_ti

#The binding free energy using mbar and ti is?
print(DG_bind_mbar, DG_bind_ti)

In [ ]:
#2. Re run adding the --subsampling option. What is the binding free energy now?
!~/sire_2018/sire.app/bin/analyse_freenrg mbar -i subsampling/bound/run001/discharge/output/lambda-*/simfile.dat -o bound_discharge.dat --subsampling
#bound phase -vanishing
!~/sire_2018/sire.app/bin/analyse_freenrg mbar -i subsampling/bound/run001/vanish/output/lambda-*/simfile.dat -o bound_vanish.dat --subsampling
#solvated phase -discharging 
!~/sire_2018/sire.app/bin/analyse_freenrg mbar -i subsampling/free/run001/discharge/output/lambda-*/simfile.dat -o free_discharge.dat --subsampling
#solvated phase -vanishing
!~/sire_2018/sire.app/bin/analyse_freenrg mbar -i subsampling/free/run001/vanish/output/lambda-*/simfile.dat -o free_vanish.dat --subsampling


In [ ]:
#Extract the free enery change from the output files
bound_discharge_mbar, bound_discharge_ti = get_free_energy('bound_discharge.dat')
bound_vanish_mbar,bound_vanish_ti = get_free_energy('bound_vanish.dat')

free_discharge_mbar,free_discharge_ti = get_free_energy('free_discharge.dat')
free_vanish_mbar, free_vanish_ti = get_free_energy('free_vanish.dat')

#mbar
DG_bound_mbar =bound_discharge_mbar + bound_vanish_mbar
DG_free_mbar = free_discharge_mbar + free_vanish_mbar
#ti
DG_bound_ti = bound_discharge_ti + bound_vanish_ti
DG_free_ti = free_discharge_ti + free_vanish_ti


#DGbind
DG_bind_mbar = DG_bound_mbar - DG_free_mbar
DG_bind_ti =  DG_bound_ti - DG_free_ti

#The binding free energy using mbar and ti is?
print(DG_bind_mbar, DG_bind_ti)

In [ ]:
#3. Try to re run using the --discard option. What do you notice?
!~/sire_2018/sire.app/bin/analyse_freenrg mbar -i subsampling/bound/run001/discharge/output/lambda-*/simfile.dat -o bound_discharge.dat --subsampling  --percent 82
#bound phase -vanishing
!~/sire_2018/sire.app/bin/analyse_freenrg mbar -i subsampling/bound/run001/vanish/output/lambda-*/simfile.dat -o bound_vanish.dat --subsampling  --percent 82
#solvated phase -discharging 
!~/sire_2018/sire.app/bin/analyse_freenrg mbar -i subsampling/free/run001/discharge/output/lambda-*/simfile.dat -o free_discharge.dat --subsampling  --percent 82
#solvated phase -vanishing
!~/sire_2018/sire.app/bin/analyse_freenrg mbar -i subsampling/free/run001/vanish/output/lambda-*/simfile.dat -o free_vanish.dat --subsampling  --percent 82



In [ ]:
#Extract the free enery change from the output files
bound_discharge_mbar, bound_discharge_ti = get_free_energy('bound_discharge.dat')
bound_vanish_mbar,bound_vanish_ti = get_free_energy('bound_vanish.dat')

free_discharge_mbar,free_discharge_ti = get_free_energy('free_discharge.dat')
free_vanish_mbar, free_vanish_ti = get_free_energy('free_vanish.dat')

#mbar
DG_bound_mbar =bound_discharge_mbar + bound_vanish_mbar
DG_free_mbar = free_discharge_mbar + free_vanish_mbar
#ti
DG_bound_ti = bound_discharge_ti + bound_vanish_ti
DG_free_ti = free_discharge_ti + free_vanish_ti


#DGbind
DG_bind_mbar = DG_free_mbar - DG_bound_mbar
DG_bind_ti =  DG_free_ti - DG_bound_ti

#The binding free energy using mbar and ti is?
print(DG_bind_mbar, DG_bind_ti)


Congratulations you have finished this tutorial! Time for a coffee or tea break :-)